# Use MCP in DSPy

This tutorial demonstrates how to integrate Model Context Protocol (MCP) with DSPy for enhanced AI applications.

## What is MCP?

Model Context Protocol (MCP) is a protocol for enabling AI models to securely access external tools and data sources. It provides a standardized way for AI applications to interact with various services while maintaining security and privacy.

## Key Features:
- Secure tool access
- Standardized protocol
- Easy integration with DSPy
- Extensible architecture

In [ ]:
# Install required packages
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import dspy
except ImportError:
    install_package("dspy")
    import dspy

try:
    import mcp
except ImportError:
    install_package("mcp")
    import mcp

import os
from typing import List, Dict, Any
import json

## Setting up MCP Client

First, let's set up an MCP client to connect to external tools and services.

In [ ]:
# Configure DSPy with your preferred language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
dspy.configure(lm=lm)

# Example MCP configuration
class MCPClient:
    def __init__(self, server_url: str = None):
        self.server_url = server_url or "http://localhost:8080"
        self.tools = {}
        
    def register_tool(self, name: str, tool_config: Dict[str, Any]):
        """Register a tool with the MCP client"""
        self.tools[name] = tool_config
        
    def call_tool(self, tool_name: str, **kwargs) -> str:
        """Call a registered tool"""
        if tool_name not in self.tools:
            return f"Tool {tool_name} not found"
        
        # Simulate tool execution
        tool_config = self.tools[tool_name]
        return f"Executed {tool_name} with args: {kwargs}"

# Initialize MCP client
mcp_client = MCPClient()

# Register some example tools
mcp_client.register_tool("file_reader", {
    "description": "Read files from the filesystem",
    "parameters": ["file_path"]
})

mcp_client.register_tool("web_search", {
    "description": "Search the web for information",
    "parameters": ["query", "num_results"]
})

mcp_client.register_tool("calculator", {
    "description": "Perform mathematical calculations",
    "parameters": ["expression"]
})

print("MCP client initialized with tools:", list(mcp_client.tools.keys()))

## Creating DSPy Modules with MCP Integration

Let's create DSPy modules that can leverage MCP tools.

In [ ]:
class ToolCallSignature(dspy.Signature):
    """Determine which tool to use and with what parameters for a given task."""
    
    task: str = dspy.InputField(desc="The task to be performed")
    available_tools: str = dspy.InputField(desc="List of available tools and their descriptions")
    tool_name: str = dspy.OutputField(desc="Name of the tool to use")
    tool_parameters: str = dspy.OutputField(desc="Parameters to pass to the tool as JSON")

class TaskExecutionSignature(dspy.Signature):
    """Execute a task using tool results and provide a final answer."""
    
    task: str = dspy.InputField(desc="The original task")
    tool_result: str = dspy.InputField(desc="Result from the tool execution")
    answer: str = dspy.OutputField(desc="Final answer based on tool result")

class MCPEnabledAgent(dspy.Module):
    """An agent that can use MCP tools to complete tasks."""
    
    def __init__(self, mcp_client: MCPClient):
        super().__init__()
        self.mcp_client = mcp_client
        self.tool_selector = dspy.ChainOfThought(ToolCallSignature)
        self.task_executor = dspy.ChainOfThought(TaskExecutionSignature)
    
    def forward(self, task: str) -> dspy.Prediction:
        # Prepare available tools information
        tools_info = "\n".join([
            f"- {name}: {config['description']}"
            for name, config in self.mcp_client.tools.items()
        ])
        
        # Select appropriate tool
        tool_selection = self.tool_selector(
            task=task,
            available_tools=tools_info
        )
        
        try:
            # Parse tool parameters
            tool_params = json.loads(tool_selection.tool_parameters)
        except json.JSONDecodeError:
            tool_params = {}
        
        # Execute the tool
        tool_result = self.mcp_client.call_tool(
            tool_selection.tool_name,
            **tool_params
        )
        
        # Generate final answer
        execution = self.task_executor(
            task=task,
            tool_result=tool_result
        )
        
        return dspy.Prediction(
            answer=execution.answer,
            tool_used=tool_selection.tool_name,
            tool_result=tool_result
        )

# Initialize the MCP-enabled agent
agent = MCPEnabledAgent(mcp_client)
print("MCP-enabled agent created successfully!")

## Testing the MCP Integration

Let's test our MCP-enabled agent with different types of tasks.

In [ ]:
# Test different tasks
test_tasks = [
    "Calculate the square root of 144",
    "Search for information about artificial intelligence",
    "Read the contents of a configuration file"
]

for task in test_tasks:
    print(f"\nTask: {task}")
    print("-" * 50)
    
    try:
        result = agent(task=task)
        print(f"Tool used: {result.tool_used}")
        print(f"Tool result: {result.tool_result}")
        print(f"Final answer: {result.answer}")
    except Exception as e:
        print(f"Error: {e}")

## Advanced MCP Features

Let's explore more advanced MCP features like tool chaining and error handling.

In [ ]:
class AdvancedMCPAgent(dspy.Module):
    """Advanced agent with tool chaining and error handling."""
    
    def __init__(self, mcp_client: MCPClient):
        super().__init__()
        self.mcp_client = mcp_client
        self.planner = dspy.ChainOfThought(
            "task -> plan: Break down the task into steps that can be executed with available tools"
        )
        self.tool_selector = dspy.ChainOfThought(ToolCallSignature)
        self.result_synthesizer = dspy.ChainOfThought(
            "task, steps_results -> final_answer: Synthesize results from multiple tool calls"
        )
    
    def forward(self, task: str) -> dspy.Prediction:
        # Create execution plan
        plan = self.planner(task=task)
        
        steps_results = []
        tools_info = "\n".join([
            f"- {name}: {config['description']}"
            for name, config in self.mcp_client.tools.items()
        ])
        
        # Extract individual steps (simplified parsing)
        steps = plan.plan.split('\n')
        
        for i, step in enumerate(steps[:3]):  # Limit to 3 steps
            if step.strip():
                try:
                    # Select tool for this step
                    tool_selection = self.tool_selector(
                        task=step.strip(),
                        available_tools=tools_info
                    )
                    
                    # Execute tool
                    try:
                        tool_params = json.loads(tool_selection.tool_parameters)
                    except json.JSONDecodeError:
                        tool_params = {}
                    
                    tool_result = self.mcp_client.call_tool(
                        tool_selection.tool_name,
                        **tool_params
                    )
                    
                    steps_results.append({
                        'step': step.strip(),
                        'tool': tool_selection.tool_name,
                        'result': tool_result
                    })
                    
                except Exception as e:
                    steps_results.append({
                        'step': step.strip(),
                        'tool': 'error',
                        'result': f'Error: {str(e)}'
                    })
        
        # Synthesize final result
        synthesis = self.result_synthesizer(
            task=task,
            steps_results=json.dumps(steps_results, indent=2)
        )
        
        return dspy.Prediction(
            final_answer=synthesis.final_answer,
            plan=plan.plan,
            steps_results=steps_results
        )

# Test advanced agent
advanced_agent = AdvancedMCPAgent(mcp_client)

complex_task = "Research AI trends, calculate market size, and prepare a summary report"
result = advanced_agent(task=complex_task)

print(f"Task: {complex_task}")
print(f"\nPlan: {result.plan}")
print(f"\nSteps executed: {len(result.steps_results)}")
print(f"\nFinal answer: {result.final_answer}")

## Security and Best Practices

When using MCP in production, consider these security and best practices:

In [ ]:
class SecureMCPClient:
    """A more secure MCP client with validation and rate limiting."""
    
    def __init__(self, server_url: str = None, api_key: str = None):
        self.server_url = server_url or "http://localhost:8080"
        self.api_key = api_key
        self.tools = {}
        self.rate_limits = {}
        self.security_policies = {
            'max_file_size': 10 * 1024 * 1024,  # 10MB
            'allowed_file_types': ['.txt', '.json', '.csv'],
            'blocked_domains': ['malicious.com'],
            'max_requests_per_minute': 60
        }
    
    def validate_tool_call(self, tool_name: str, **kwargs) -> bool:
        """Validate tool call against security policies."""
        
        # Check if tool exists
        if tool_name not in self.tools:
            return False
        
        # Validate file operations
        if tool_name == 'file_reader' and 'file_path' in kwargs:
            file_path = kwargs['file_path']
            
            # Check file extension
            if not any(file_path.endswith(ext) for ext in self.security_policies['allowed_file_types']):
                return False
        
        # Validate web searches
        if tool_name == 'web_search' and 'query' in kwargs:
            query = kwargs['query'].lower()
            
            # Check for blocked domains
            if any(domain in query for domain in self.security_policies['blocked_domains']):
                return False
        
        return True
    
    def call_tool_secure(self, tool_name: str, **kwargs) -> str:
        """Securely call a tool with validation."""
        
        # Validate the call
        if not self.validate_tool_call(tool_name, **kwargs):
            return "Tool call blocked by security policy"
        
        # Check rate limits (simplified)
        import time
        current_time = time.time()
        if tool_name in self.rate_limits:
            if current_time - self.rate_limits[tool_name] < 1:  # 1 second cooldown
                return "Rate limit exceeded"
        
        self.rate_limits[tool_name] = current_time
        
        # Execute tool (simulated)
        return f"Securely executed {tool_name} with args: {kwargs}"

# Example usage
secure_client = SecureMCPClient(api_key="your-secure-api-key")
secure_client.tools = mcp_client.tools  # Copy tools

# Test secure calls
print("Secure tool calls:")
print(secure_client.call_tool_secure("file_reader", file_path="data.txt"))
print(secure_client.call_tool_secure("file_reader", file_path="malware.exe"))  # Should be blocked
print(secure_client.call_tool_secure("web_search", query="AI trends"))

## Conclusion

This tutorial demonstrated how to integrate MCP with DSPy to create powerful AI applications that can securely access external tools and services. Key takeaways:

1. **MCP Integration**: Easy to integrate MCP clients with DSPy modules
2. **Tool Selection**: Use DSPy's reasoning capabilities to select appropriate tools
3. **Advanced Features**: Support for tool chaining and complex workflows
4. **Security**: Implement proper validation and security measures
5. **Best Practices**: Rate limiting, input validation, and error handling

## Next Steps

- Explore real MCP server implementations
- Implement custom tools for your specific use cases
- Add monitoring and logging for production deployments
- Consider using DSPy optimizers to improve tool selection accuracy